In [1]:
import os, sys 
from pathlib import Path
from os.path import dirname, realpath
script_dir = Path(dirname(realpath('.')))
module_dir = str(script_dir)
sys.path.insert(0, module_dir + '/modules')
import numpy as np
import utility as ut
import matplotlib.pyplot as plt
import pandas as pd
import rfm, skipRFM, deepSkip, localSkip_2_2, localSkip_8_1, localDeepSkip_2_2, localDeepSkip_8_1
import torch
from torch import nn
import l63, l96, ks
import config

repo, config_id = '..', 'config_1'
device = "cuda" if torch.cuda.is_available() else "cpu"
L0, L1, placeholder_beta = 0.4, 3.5, 7e-8
data_folder = f"{repo}/data"

dynamical_systems = ['KS']#, 'L96', 'KS']
ds_modules = {'L63': l63, 'L96': l96, 'KS': ks}
arch_modules = {'RFM': rfm, 'SkipRFM': skipRFM, 'DeepSkip': deepSkip, 'LocalSkip_2_2': localSkip_2_2, 'LocalSkip_8_1': localSkip_8_1,\
                'LocalDeepSkip_2_2': localDeepSkip_2_2, 'LocalDeepSkip_8_1': localDeepSkip_8_1}
for dynamical_system in dynamical_systems:
    print(f"Working on {dynamical_system}: generating training and testing data ...")
    # generate training and testing data
    setup = getattr(config, config_id)(dynamical_system)
    train, test = ds_modules[dynamical_system].gen_data(**setup['data_gen'])
    train, test = torch.tensor(train, device=device), torch.tensor(test, device=device)
    # set up architecture and test beta
    for architecture in setup['beta_arch']:
        print(f"Working on {dynamical_system}-{architecture} ...")
        D_r_list, B_list = list(zip(*setup['beta_arch'][architecture]))
        drf_args = [D_r_list[0], B_list[0], L0, L1, train, placeholder_beta, architecture,\
                    f"{data_folder}/{dynamical_system}/{config_id}/{architecture}/beta", False]
        beta_tester = rfm.BetaTester(arch_modules[architecture].DeepRF, D_r_list, B_list, [train]*len(D_r_list), test, *drf_args)
        beta_tester.search_beta(**{**setup['beta'], **setup['prediction_time']})

Working on KS: generating training and testing data ...


/Users/pman0581/Documents/GitHub/DeepRFM/modules/ks.py:66: ComplexWarning: Casting complex values to real discards the imaginary part
  self.vv[0,:] = self.v0
/Users/pman0581/Documents/GitHub/DeepRFM/modules/ks.py:194: ComplexWarning: Casting complex values to real discards the imaginary part
  self.vv[self.ioutnum,:] = self.v


Time taken by gen_data is 18.2341 seconds
Working on KS-LocalSkip_8_1 ...
Estimated time to find optimal beta for (D_r, B) = (512, 1) is 0.15 hours
Estimated time to find optimal beta for (D_r, B) = (1024, 1) is 0.21 hours
Estimated time to find optimal beta for (D_r, B) = (2048, 1) is 0.33 hours
Estimated time to find optimal beta for (D_r, B) = (4096, 1) is 1.09 hours
Estimated time to find optimal beta for (D_r, B) = (8192, 1) is 4.24 hours
Estimated time to find optimal beta for (D_r, B) = (16384, 1) is 57.45 hours
Estimated time to find optimal beta for all (D_r, B) for current architecture is 63.47 hours
Experiments for (D_r, B, beta) = (512, 1, 1.00E-11) took 6.14E+00s
Experiments for (D_r, B, beta) = (512, 1, 1.36E-11) took 5.86E+00s
Experiments for (D_r, B, beta) = (512, 1, 1.72E-11) took 5.86E+00s
Experiments for (D_r, B, beta) = (512, 1, 2.08E-11) took 5.89E+00s
Experiments for (D_r, B, beta) = (512, 1, 2.44E-11) took 6.00E+00s


KeyboardInterrupt: 